## This Notebook is for training different ML models on TF-IDF 

In [1]:
import pickle
import warnings
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.utils import shuffle
from molearn.classifiers.BR import BR
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from molearn.classifiers.Ensemble import Ensemble
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import classification_report,f1_score
from molearn.classifiers.classifier_chains import CC,RCC,MCC

In [2]:
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"

### Define some useful methods

In [3]:
def Hamming_loss(Ytest,Ypred):
    ''' Hamming loss aka Hamming distance '''
    return 1.-Hamming_score(Ytest,Ypred)

def Hamming_score(Ytest,Ypred):
    ''' Hamming score aka Hamming match '''
    N_test,L = Ytest.shape
    return np.sum((Ytest == Ypred) * 1.) / N_test / L

def Hamming_matches(Ytest,Ypred):
    N_test,L = Ytest.shape
    return np.sum((Ytest == Ypred) * 1.,axis=0) / N_test 

def Hamming_losses(Ytest,Ypred):
    return 1.-Hamming_matches(Ytest,Ypred)

def Exact_match(Ytest,Ypred):
    N_test,L = Ytest.shape
    return np.sum(np.sum((Ytest == Ypred) * 1,axis=1)==L) * 1. / N_test

In [4]:
def grid_search(train_x, train_y, test_x, test_y, parameters, pipeline):
    grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1, verbose=10)
    grid_search_tune.fit(train_x, train_y)

    print
    print("Best parameters set:")
    print (grid_search_tune.best_estimator_.steps)
    print

    # measuring performance on test set
    print ("Applying best classifier on test data:")
    best_clf = grid_search_tune.best_estimator_
    
    predictions = best_clf.predict(test_x)
    print('grid_search_tune.best_estimator : ',grid_search_tune.best_estimator_.steps[0])
    return best_clf

In [5]:
## save model
def save_model(clf,filename,folder):
    pickle.dump(clf, open("../trained-model/"+folder+"/"+filename, 'wb'))

## classification methods

In [6]:
def logisticRegression(x_train, x_test, y_train, y_test):
        
    print ("LogisticRegression")
    pipeline = Pipeline([
        ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
    ])
    parameters = {
        "clf__estimator__C": [0.1,1,10],
        "clf__estimator__class_weight": [None],
    }
    clf = grid_search(x_train, y_train, x_test, y_test, parameters, pipeline)
    return clf

In [7]:
def adaboost(x_train, x_test, y_train, y_test):
        
    print ("LogisticRegression")
    pipeline = Pipeline([
        ('clf', OneVsRestClassifier(AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    algorithm="SAMME"), n_jobs=-1)),
    ])
    parameters = {
        "clf__estimator__learning_rate": [1,1.5],
        "clf__estimator__n_estimators": [6],
    }
    clf = grid_search(x_train, y_train, x_test, y_test, parameters, pipeline)
    return clf

In [8]:
def naivebayes(x_train, x_test, y_train, y_test):
        
    print ("LogisticRegression")
    pipeline = Pipeline([
        ('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None), n_jobs=-1)),
    ])
    parameters = {
        'clf__estimator__alpha': (1e-2, 1e-3)
    }
    clf = grid_search(x_train, y_train, x_test, y_test, parameters, pipeline)
    return clf

In [9]:
def svm(x_train, x_test, y_train, y_test):


    print ("LinearSVC")
    pipeline = Pipeline([
        ('clf', OneVsRestClassifier(SVC(), n_jobs=-1)),
    ])
    
    parameters ={
         'clf__estimator__kernel': ['rbf'],
         'clf__estimator__gamma': [1e-3],
         'clf__estimator__C': [10]
        }
        
    clf = grid_search(x_train, y_train, x_test, y_test, parameters, pipeline)    
    return clf

## 1.TF-IDF(bag-of-words)

### 1.1 Reading The data

In [40]:
x_train = pd.read_csv('../vectors/all/tfidf/x_data_train_tfidf.csv')
y_train = pd.read_csv('../vectors/all/tfidf/y_data_train_tfidf.csv')
x_test = pd.read_csv('../vectors/all/tfidf/x_data_test_tfidf.csv')
y_test = pd.read_csv('../vectors/all/tfidf/y_data_test_tfidf.csv')

In [41]:
x_train = shuffle(x_train.as_matrix())
y_train = shuffle(y_train.as_matrix())
x_test = shuffle(x_test.as_matrix())
y_test = shuffle(y_test.as_matrix())


In [42]:
x_train = x_train[0:600]
y_train = y_train[0:600]
x_test = x_test[0:100]
y_test = y_test[0:100]
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (600, 51053)
y_train shape:  (600, 28)
x_test shape:  (100, 51053)
y_test shape:  (100, 28)


#### 1.3 shape of train and test

In [44]:
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (600, 51053)
y_train shape:  (600, 28)
x_test shape:  (100, 51053)
y_test shape:  (100, 28)


### 1.4 apply different algorithm

#### 1.4.1 Logistic regression

##### 1.4.1.1 OneVsRestClassifier

In [45]:
bst_clf_logisticregression_OneVsRestClassifier_tfidf=logisticRegression(x_train, x_test, y_train, y_test)
save_model(bst_clf_logisticregression_OneVsRestClassifier_tfidf,'logisticregression_OneVsRestClassifier_tfidf.sav','tfidf')
prediction_logisticregression_OneVsRestClassifier_tfidf = bst_clf_logisticregression_OneVsRestClassifier_tfidf.predict(x_test)

LogisticRegression
Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] clf__estimator__C=0.1, clf__estimator__class_weight=None ........
[CV] clf__estimator__C=0.1, clf__estimator__class_weight=None ........
[CV] clf__estimator__C=1, clf__estimator__class_weight=None ..........
[CV] clf__estimator__C=1, clf__estimator__class_weight=None ..........
[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, score=0.0, total= 3.4min
[CV] clf__estimator__C=10, clf__estimator__class_weight=None .........
[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, score=0.0, total= 3.5min
[CV] clf__estimator__C=10, clf__estimator__class_weight=None .........


[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:  3.5min remaining:  7.0min


[CV]  clf__estimator__C=1, clf__estimator__class_weight=None, score=0.0, total= 3.6min


[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  3.6min remaining:  3.6min


[CV]  clf__estimator__C=1, clf__estimator__class_weight=None, score=0.0, total= 3.6min


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:  3.6min remaining:  1.8min


[CV]  clf__estimator__C=10, clf__estimator__class_weight=None, score=0.0033333333333333335, total= 3.1min
[CV]  clf__estimator__C=10, clf__estimator__class_weight=None, score=0.016666666666666666, total= 3.1min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  6.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  6.7min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=-1))]
Applying best classifier on test data:
grid_search_tune.best_estimator :  ('clf', OneVsRestClassifier(estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=-1))


##### 1.4.1.2 ClassifierChain

In [ ]:
chains = [ClassifierChain(LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False), order='random', random_state=i)
          for i in range(1)]

for idx,chain in enumerate(chains):
    if idx%4==0 or idx==10:
        print(idx,"chain")
    chain.fit(x_train, y_train)

In [ ]:
y_pred_chains = np.array([chain.predict(x_test) for chain in
                          chains])
predict_logisticregression_ensemble_ClassifierChain_tfidf = y_pred_chains.mean(axis=0)

temp = []
org = []
for i in range(len(predict_logisticregression_ensemble_ClassifierChain_tfidf)):
    for j in range(len(predict_logisticregression_ensemble_ClassifierChain_tfidf[0])):
        temp.append(int(predict_logisticregression_ensemble_ClassifierChain_tfidf[i][j]+0.8))
    org.append(temp)
    temp = []
    
predict_logisticregression_ensemble_ClassifierChain_tfidf = np.asarray(org) 

In [ ]:
cc = CC(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
cc.fit(x_train,y_train)
save_model(cc,'LogisticRegression_tfidf_CC.sav','tfidf')

In [ ]:
predict_logisticregression_CC_ClassifierChain_tfidf=cc.predict(x_test)

In [ ]:
br = BR(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
br.fit(x_train,y_train)
save_model(br,'LogisticRegression_tfidf_BR.sav','tfidf')

In [ ]:
predict_logisticregression_BR_ClassifierChain_tfidf=br.predict(x_test)

In [ ]:
mcc = MCC(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
mcc.fit(x_train,y_train)
save_model(mcc,'LogisticRegression_tfidf_MCC.sav','tfidf')

In [ ]:
predict_logisticregression_MCC_ClassifierChain_tfidf=mcc.predict(x_test)

##### 1.4.1.3 visualization and scores

In [ ]:
predict_logisticregression_ensemble_ClassifierChain_tfidf.shape

In [ ]:
f1_score_prediction_logisticregression_OneVsRestClassifier_tfidf = f1_score(y_test,prediction_logisticregression_OneVsRestClassifier_tfidf,average='weighted')
f1_score_prediction_logisticregression_pure_ClassifierChain_tfidf = f1_score(y_test,predict_logisticregression_ensemble_ClassifierChain_tfidf,average='weighted')
f1_score_logisticregression_CC_ClassifierChain_tfidf = f1_score(y_test,predict_logisticregression_CC_ClassifierChain_tfidf,average='weighted')
f1_score_logisticregression_BR_ClassifierChain_tfidf = f1_score(y_test,predict_logisticregression_BR_ClassifierChain_tfidf,average='weighted')
f1_score_logisticregression_MCC_ClassifierChain_tfidf = f1_score(y_test,predict_logisticregression_MCC_ClassifierChain_tfidf,average='weighted')

print('f1_score_prediction_logisticregression_OneVsRestClassifier_tfidf: ',f1_score_prediction_logisticregression_OneVsRestClassifier_tfidf)
print('f1_score_prediction_logisticregression_pure_chain_tfidf: ',f1_score_prediction_logisticregression_pure_ClassifierChain_tfidf)
print('f1_score_logisticregression_CC_ClassifierChain_tfidf: ',f1_score_logisticregression_CC_ClassifierChain_tfidf)
print('f1_score_logisticregression_BR_ClassifierChain_tfidf: ',f1_score_logisticregression_BR_ClassifierChain_tfidf)
print('f1_score_logisticregression_MCC_ClassifierChain_tfidf: ',f1_score_logisticregression_MCC_ClassifierChain_tfidf)

In [ ]:
Hamming_loss_prediction_logisticregression_OneVsRestClassifier_tfidf = Hamming_loss(y_test,prediction_logisticregression_OneVsRestClassifier_tfidf)
Hamming_loss_prediction_logisticregression_pure_ClassifierChain_tfidf = Hamming_loss(y_test,predict_logisticregression_ensemble_ClassifierChain_tfidf)
Hamming_loss_logisticregression_CC_ClassifierChain_tfidf = Hamming_loss(y_test,predict_logisticregression_CC_ClassifierChain_tfidf)
Hamming_loss_logisticregression_BR_ClassifierChain_tfidf = Hamming_loss(y_test,predict_logisticregression_BR_ClassifierChain_tfidf)
Hamming_loss_logisticregression_MCC_ClassifierChain_tfidf = Hamming_loss(y_test,predict_logisticregression_MCC_ClassifierChain_tfidf)

print('Hamming_loss_prediction_logisticregression_OneVsRestClassifier_tfidf: ',Hamming_loss_prediction_logisticregression_OneVsRestClassifier_tfidf)
print('Hamming_loss_prediction_logisticregression_pure_chain_tfidf: ',Hamming_loss_prediction_logisticregression_pure_ClassifierChain_tfidf)
print('Hamming_loss_logisticregression_CC_ClassifierChain_tfidf: ',Hamming_loss_logisticregression_CC_ClassifierChain_tfidf)
print('Hamming_loss_logisticregression_BR_ClassifierChain_tfidf: ',Hamming_loss_logisticregression_BR_ClassifierChain_tfidf)
print('Hamming_loss_logisticregression_MCC_ClassifierChain_tfidf: ',Hamming_loss_logisticregression_MCC_ClassifierChain_tfidf)

In [ ]:
Hamming_score_prediction_logisticregression_OneVsRestClassifier_tfidf = Hamming_score(y_test,prediction_logisticregression_OneVsRestClassifier_tfidf)
Hamming_score_prediction_logisticregression_pure_ClassifierChain_tfidf = Hamming_score(y_test,predict_logisticregression_ensemble_ClassifierChain_tfidf)
Hamming_score_logisticregression_CC_ClassifierChain_tfidf = Hamming_score(y_test,predict_logisticregression_CC_ClassifierChain_tfidf)
Hamming_score_logisticregression_BR_ClassifierChain_tfidf = Hamming_score(y_test,predict_logisticregression_BR_ClassifierChain_tfidf)
Hamming_score_logisticregression_MCC_ClassifierChain_tfidf = Hamming_score(y_test,predict_logisticregression_MCC_ClassifierChain_tfidf)

print('Hamming_score_prediction_logisticregression_OneVsRestClassifier_tfidf: ',Hamming_score_prediction_logisticregression_OneVsRestClassifier_tfidf)
print('Hamming_score_prediction_logisticregression_pure_chain_tfidf: ',Hamming_score_prediction_logisticregression_pure_ClassifierChain_tfidf)
print('Hamming_score_logisticregression_CC_ClassifierChain_tfidf: ',Hamming_score_logisticregression_CC_ClassifierChain_tfidf)
print('Hamming_score_logisticregression_BR_ClassifierChain_tfidf: ',Hamming_score_logisticregression_BR_ClassifierChain_tfidf)
print('Hamming_score_logisticregression_MCC_ClassifierChain_tfidf: ',Hamming_score_logisticregression_MCC_ClassifierChain_tfidf)

In [ ]:
Exact_match_prediction_logisticregression_OneVsRestClassifier_tfidf = Exact_match(y_test,prediction_logisticregression_OneVsRestClassifier_tfidf)
Exact_match_prediction_logisticregression_pure_ClassifierChain_tfidf = Exact_match(y_test,predict_logisticregression_ensemble_ClassifierChain_tfidf)
Exact_match_logisticregression_CC_ClassifierChain_tfidf = Exact_match(y_test,predict_logisticregression_CC_ClassifierChain_tfidf)
Exact_match_logisticregression_BR_ClassifierChain_tfidf = Exact_match(y_test,predict_logisticregression_BR_ClassifierChain_tfidf)
Exact_match_logisticregression_MCC_ClassifierChain_tfidf = Exact_match(y_test,predict_logisticregression_MCC_ClassifierChain_tfidf)

print('Exact_match_prediction_logisticregression_OneVsRestClassifier_tfidf: ',Exact_match_prediction_logisticregression_OneVsRestClassifier_tfidf)
print('Exact_match_prediction_logisticregression_pure_chain_tfidf: ',Exact_match_prediction_logisticregression_pure_ClassifierChain_tfidf)
print('Exact_match_logisticregression_CC_ClassifierChain_tfidf: ',Exact_match_logisticregression_CC_ClassifierChain_tfidf)
print('Exact_match_logisticregression_BR_ClassifierChain_tfidf: ',Exact_match_logisticregression_BR_ClassifierChain_tfidf)
print('Exact_match_logisticregression_MCC_ClassifierChain_tfidf: ',Exact_match_logisticregression_MCC_ClassifierChain_tfidf)

#### 1.4.2 DecisionTreeClassifier

In [47]:
DecisionTreeClassifier_tfidf = DecisionTreeClassifier(min_samples_split=2, random_state=1)
DecisionTreeClassifier_tfidf.fit(x_train,y_train)
save_model(DecisionTreeClassifier_tfidf,'DecisionTreeClassifier_tfidf.sav','tfidf')

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [48]:
predict_DecisionTreeClassifier_tfidf = DecisionTreeClassifier_tfidf.predict(x_test)

In [49]:
f1_score_DecisionTreeClassifier_tfidf = f1_score(y_test,predict_DecisionTreeClassifier_tfidf,average='weighted')

print('f1_score_prediction_DecisionTreeClassifier_tfidf: ',f1_score_DecisionTreeClassifier_tfidf)

f1_score_prediction_DecisionTreeClassifier_tfidf:  0.262341236935


In [50]:
Hamming_loss_DecisionTreeClassifier_tfidf = Hamming_loss(y_test,predict_DecisionTreeClassifier_tfidf)

print('Hamming_loss_DecisionTreeClassifier_tfidf: ',Hamming_loss_DecisionTreeClassifier_tfidf)


Hamming_loss_DecisionTreeClassifier_tfidf:  0.111428571429


In [51]:
Hamming_score_DecisionTreeClassifier_tfidf = Hamming_score(y_test,predict_DecisionTreeClassifier_tfidf)

print('Hamming_score_DecisionTreeClassifier_tfidf: ',Hamming_score_DecisionTreeClassifier_tfidf)


Hamming_score_DecisionTreeClassifier_tfidf:  0.888571428571


In [53]:
Exact_match_DecisionTreeClassifier_tfidf = Exact_match(y_test,predict_DecisionTreeClassifier_tfidf)

print('Exact_match_DecisionTreeClassifier_tfidf: ',Exact_match_DecisionTreeClassifier_tfidf)


Exact_match_DecisionTreeClassifier_tfidf:  0.03


In [54]:
from keras import backend as K

# bp mll loss function
# y_true, y_pred must be 2D tensors of shape (batch dimension, number of labels)
# y_true must satisfy y_true[i][j] == 1 iff sample i has label j
def bp_mll_loss(y_true, y_pred):
 
    # get true and false labels
    y_i = K.equal(y_true, K.ones_like(y_true))
    y_i_bar = K.not_equal(y_true, K.ones_like(y_true))
    
    # cast to float as keras backend has no logical and
    y_i = K.cast(y_i, dtype='float32')
    y_i_bar = K.cast(y_i_bar, dtype='float32')

    # get indices to check
    truth_matrix = pairwise_and(y_i, y_i_bar)

    # calculate all exp'd differences
    sub_matrix = pairwise_sub(y_pred, y_pred)
    exp_matrix = K.exp(-sub_matrix)

    # check which differences to consider and sum them
    sparse_matrix = exp_matrix * truth_matrix
    sums = K.sum(sparse_matrix, axis=[1,2])

    # get normalizing terms and apply them
    y_i_sizes = K.sum(y_i, axis=1)
    y_i_bar_sizes = K.sum(y_i_bar, axis=1)
    normalizers = y_i_sizes * y_i_bar_sizes
    results = sums / normalizers

    # sum over samples
    return K.sum(results)


# compute pairwise differences between elements of the tensors a and b
def pairwise_sub(a, b):
    column = K.expand_dims(a, 2)
    row = K.expand_dims(b, 1)
    return column - row

# compute pairwise logical and between elements of the tensors a and b
def pairwise_and(a, b):
    column = K.expand_dims(a, 2)
    row = K.expand_dims(b, 1)
    return K.minimum(column, row)


Using TensorFlow backend.


In [55]:
n = x_train.shape[0]
dim_no = x_train.shape[1]
class_no = y_train.shape[1]

In [56]:
from keras.models import Sequential
from keras.layers import Dense, Dropout



# create simple mlp
model = Sequential()
model.add(Dense(300, input_dim=dim_no, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.6))
model.add(Dense(128, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.6))
model.add(Dense(64, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(class_no, activation='sigmoid', kernel_initializer='glorot_uniform'))
model.compile(loss=bp_mll_loss, optimizer='adagrad', metrics=[])

# train a few epochs
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
600/600 [==============================] - 5s 8ms/step - loss: 24.9179
Epoch 2/5
600/600 [==============================] - 4s 6ms/step - loss: 22.4449
Epoch 3/5
600/600 [==============================] - 4s 6ms/step - loss: 22.0011
Epoch 4/5
600/600 [==============================] - 4s 6ms/step - loss: 21.7054
Epoch 5/5
600/600 [==============================] - 4s 6ms/step - loss: 21.4001


In [58]:

bpmll_Word2Vec = model.predict(x_test)


#report =classification_report(Y_test, bpmll_Word2Vec)
print (len(bpmll_Word2Vec))

print (bpmll_Word2Vec.shape)

b = bpmll_Word2Vec[2][0]

a = bpmll_Word2Vec

res = []
tmp = []
for j in range(bpmll_Word2Vec.shape[0]):
	for i in range(bpmll_Word2Vec.shape[1]):
		tmp.append(int(bpmll_Word2Vec[j][i]+0.01))
	res.append(tmp)
	tmp = []	

print(bpmll_Word2Vec[2])

print(a)
print(y_test[2])
res = pd.DataFrame(res)
res = res.as_matrix()
print(res,'res')
report =classification_report(y_test, res)
print (report)

100
(100, 28)
[  1.07279792e-02   5.53013012e-03   1.21583402e-01   9.89946544e-01
   1.79836573e-03   3.03064415e-04   6.69351313e-03   2.30080143e-01
   3.92278701e-01   9.97813106e-01   1.79367676e-01   7.82854203e-03
   1.58002064e-01   3.92395863e-03   1.11133792e-02   9.61005449e-01
   2.86067128e-01   6.74827471e-02   1.28031686e-01   1.25051722e-01
   4.78469402e-01   1.26170516e-01   2.34149001e-03   9.96589899e-01
   4.57244478e-02   3.09328362e-03   9.35036018e-02   4.74386178e-02]
[[ 0.01380529  0.00642244  0.12232939 ...,  0.00409534  0.11350527
   0.06305631]
 [ 0.00726808  0.00329856  0.10542663 ...,  0.00181003  0.07346009
   0.03900498]
 [ 0.01072798  0.00553013  0.1215834  ...,  0.00309328  0.0935036
   0.04743862]
 ..., 
 [ 0.00600706  0.00262066  0.10390926 ...,  0.00140605  0.06460322
   0.03301838]
 [ 0.01786968  0.01000424  0.15377772 ...,  0.00623577  0.11988172
   0.0688329 ]
 [ 0.00463269  0.00193006  0.08776482 ...,  0.00101964  0.06270926
   0.03015795]]
[0 

In [59]:
print(Hamming_loss(y_test, res))
print(Hamming_score(y_test, res))
print(Exact_match(y_test, res))


0.0960714285714
0.903928571429
0.04
